# extract data at depths from a non-layerwise simulation

In [1]:
import sys
sys.path.append("../pyfesom/") # add pyfesom to search path
sys.path.append("/home/ollie/jhauck/py_fesom/modules/")
import os
import pyfesom as pf
import numpy as np
from netCDF4 import Dataset, MFDataset
from load_mesh_data_new import load_mesh

no cmocean


In [2]:
# run specifics
runname      = 'test_o2'
resultpath   = '/work/ollie/ncara/fesom/fesom-1.4-recom/'+runname+'/'
variable     = 'bgc22'
year = 1980

# load mesh
meshpath     = '/work/ollie/jhauck/input/meshes/core_new_384/'
mesh = load_mesh(meshpath,abg=[0,0,0],get3d=True)
mesh.zlevs = -mesh.zlevs # depth is made negative

usepickle =  False
abg =  [0, 0, 0]
no pickle
Load 2d part of the grid in 5 second(s)
Load 3d part of the grid in 13 seconds


In [3]:
# some helpful mesh info
print dir(mesh)
print mesh.zlevs # in a zlevel mesh, the depth levels are the same for each surface node!
print mesh.n32.shape
print mesh.n32[0,:] # nodes at depth of surface node 0

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'alpha', 'aux3dfile', 'beta', 'e2d', 'elem', 'elm2dfile', 'gamma', 'n2d', 'n32', 'n3d', 'nlev', 'no_cyclic_elem', 'nod2dfile', 'nod3dfile', 'nodes', 'path', 'read2d', 'read3d', 'reso', 'topo', 'voltri', 'x2', 'x3', 'y2', 'y3', 'z3', 'zlevs']
[    0.   -10.   -20.   -30.   -40.   -50.   -60.   -70.   -80.   -90.
  -100.  -115.  -135.  -160.  -190.  -230.  -280.  -340.  -410.  -490.
  -580.  -680.  -790.  -910. -1040. -1180. -1330. -1500. -1700. -1920.
 -2150. -2400. -2650. -2900. -3150. -3400. -3650. -3900. -4150. -4400.
 -4650. -4900. -5150. -5400. -5650. -5900.]
(126859, 47)
[      1  126860  253719  380578  503947  625098  744228  859699  973093
 1085036 1195664 1305023 1413225 1519729    -999    -999    -999    -999
    -999    -999  

In [4]:
# load variable
fl = Dataset(resultpath+variable+'_fesom_'+str(year)+'0101.nc')

# load data, calculate annual mean
data_mean = fl.variables[variable][:,:].mean(axis=0)
print data_mean.shape # one vector with all 3D nodes
print np.min(data_mean)
print np.max(data_mean)

(3668773,)
2.23e-16
364.484


In [5]:
# extract data at a list of levels 
# (if defined as mesh.zlevs, this corresponds to a re-organisation of the data to layerwise output)

depth_list = [-0,-5,-10]
#depth_list = mesh.zlevs

level_data = np.zeros((mesh.n2d,len(depth_list)))
#elem_no_nan = np.zeros((mesh.n2d,len(depth_list)))
for i in range(len(depth_list)):
    print 'Load data at '+str(depth_list[i])+'m'
    #level_data1, elem_no_nan1 = pf.get_data_sigma(data_mean,mesh,depth_list[i]) # for sigma mesh
    level_data1, elem_no_nan1 = pf.get_data(data_mean,mesh,depth_list[i])
    level_data[:,i]  = level_data1

Load data at 0m
0
For depth 0 model level 0.0 will be used
Load data at 5m
0
For depth 5 model level 0.0 will be used
Load data at 10m
0
For depth 10 model level 0.0 will be used


In [6]:
dd=1

print 'Some info of data at depth level '+str(mesh.zlevs[dd])+'m :'
print level_data[:,dd]
print 'Min '+str(np.min(level_data[:,dd]))
print 'Max '+str(np.max(level_data[:,dd]))

Some info of data at depth level -10.0m :
[ 330.78952026  322.16625977  318.37698364 ...,  333.00286865  206.09089661
  206.28025818]
Min 168.792373657
Max 363.723602295
